In [ ]:
import scvelo as scv
import scanpy as sc
import cell2fate as c2f
import pickle as pickle
import os
os.chdir('..')
os.chdir('..')
from eval_utils import cross_boundary_correctness
from datetime import datetime
import pandas as pd
import numpy as np
from os.path import exists
import matplotlib.pyplot as plt
import torch
import unitvelo as utv
method = 'Cell2fateDynamicalModel_batchSize10000epochs1000'
data_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_datasets/'
save_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_results/'
datasets = ['Pancreas_with_cc',  'DentateGyrus' , 'MouseBoneMarrow', 'MouseErythroid', 'HumanBoneMarrow']
n_genes_list = np.array((2000, 3000))
n_counts_list = np.array((10, 20))

for i in (1,0):
    for j in range(len(n_counts_list)):
        for k in (3,0,1,2,4):
            print(i)
            print(j)
            print(k)
            dataset = datasets[k]
            print(dataset)
            n_genes = n_genes_list[i]
            min_counts = n_counts_list[j]
            model_index = str(i) + '-' + str(j) + '-' + str(k)
            save_name = method + '_'
            if exists(save_dir + save_name + '_CBDC_fullBenchmark.csv'):
                tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
                if model_index in tab.index:
                    continue
            adata = sc.read_h5ad(data_dir + dataset + '/' + dataset + '_anndata.h5ad')
            adata = c2f.utils.get_training_data(adata, cells_per_cluster = 10**6, cluster_column = 'clusters',
                                            remove_clusters = [], min_shared_counts = min_counts, n_var_genes= n_genes)
            c2f.Cell2fate_DynamicalModel.setup_anndata(adata, spliced_label='spliced', unspliced_label='unspliced')    
            n_modules = c2f.utils.get_max_modules(adata)
            mod = c2f.Cell2fate_DynamicalModel(adata,
                                               n_modules = n_modules)   
            mod.train(batch_size = 10000, max_epochs = 1000)
            sample_kwarg = {"num_samples": 3, "batch_size" : adata.n_obs,
                 "use_gpu" : True, 'return_samples': False}
            adata = mod.export_posterior(adata, sample_kwargs = sample_kwarg)
            mod.compute_and_plot_total_velocity_scvelo(adata, save = False, delete = False)
            # Calculate performance metrics:
            file = open(data_dir + dataset + '/' + dataset + '_groundTruth.pickle' ,'rb')
            ground_truth = pickle.load(file)
            metrics = utv.evaluate(adata, ground_truth, 'clusters', 'velocity')
            cb_score = [np.mean(metrics['Cross-Boundary Direction Correctness (A->B)'][x])
                        for x in metrics['Cross-Boundary Direction Correctness (A->B)'].keys()]
            if exists(save_dir + save_name + '_CBDC_fullBenchmark.csv'):
                tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
            else:
                c_names = ['CBDC']
                tab = pd.DataFrame(columns = c_names)
            tab.loc[model_index, 'CBDC'] = np.mean(cb_score)
            tab.to_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv')
tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
tab.loc['AVERAGE', 'CBDC'] = np.mean(tab['CBDC'])
tab.to_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv')

Global seed set to 0


(Running UniTVelo 0.2.5)
2023-10-22 11:11:37
1
0
3
MouseErythroid
Keeping at most 1000000 cells per cluster
Filtered out 46616 genes that are detected 10 counts (shared).
Extracted 3000 highly variable genes.
Leiden clustering ...
Number of Leiden Clusters: 9
Maximal Number of Modules: 10


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sampling global variables, sample: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.51it/s]
Computing total RNAvelocity ...


  0%|          | 0/9815 [00:00<?, ?cells/s]

...

Number of Leiden Clusters: 11
Maximal Number of Modules: 12
Sampling global variables, sample: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.65it/s]
Computing total RNAvelocity ...


  0%|          | 0/5780 [00:00<?, ?cells/s]

# Cross-Boundary Direction Correctness (A->B)
{('HSC_1', 'Ery_1'): -0.18813467269075884, ('HSC_1', 'HSC_2'): 0.10976570235314081, ('Ery_1', 'Ery_2'): -0.3757368675854454}
Total Mean: -0.15136861264102114
# In-cluster Coherence
{'CLP': 0.87532306, 'DCs': 0.85812527, 'Ery_1': 0.8726341, 'Ery_2': 0.8753473, 'HSC_1': 0.8804341, 'HSC_2': 0.8871702, 'Mega': 0.8646431, 'Mono_1': 0.8823728, 'Mono_2': 0.8755601, 'Precursors': 0.8847979}
Total Mean: 0.8756407499313354
1
1
3
MouseErythroid
Keeping at most 1000000 cells per cluster
Filtered out 47456 genes that are detected 20 counts (shared).
Extracted 3000 highly variable genes.
Leiden clustering ...


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Number of Leiden Clusters: 9
Maximal Number of Modules: 10
Epoch 412/1000:  41%|██████████████████████████████████████████████████████████████▍                                                                                         | 411/1000 [04:29<06:26,  1.52it/s, v_num=1, elbo_train=3.56e+7]